# Sales Forecast Notebook
This notebook reads the `sales_forecast_dataset.csv`, performs basic EDA, fits a simple Linear Regression per customer to forecast next 6 months, and exports forecast CSVs for Tableau.
Run the cells in order. The data file should be in `data/sales_forecast_dataset.csv` relative to the notebook.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import os

# Ensure inline plots (if running in Jupyter)
%matplotlib inline


In [ ]:
# Load data
df = pd.read_csv('../data/sales_forecast_dataset.csv')
df['Month_Num'] = pd.to_datetime(df['Month'], format='%b').dt.month
df.head()


In [ ]:
# Simple EDA: plot Actual Sales trend per customer
for cust, g in df.groupby('Customer'):
    plt.figure(figsize=(8,3))
    plt.plot(g['Month_Num'], g['Actual_Sales'], marker='o', label='Actual')
    plt.title(f'Actual Sales - {cust}')
    plt.xlabel('Month')
    plt.ylabel('Sales (USD)')
    plt.grid(alpha=0.3)
    plt.show()


In [ ]:
# Forecast next 6 months per customer using Linear Regression
future_steps = 6
results = []
for cust, g in df.groupby('Customer'):
    g_sorted = g.sort_values(['Year','Month_Num'])
    X = g_sorted[['Month_Num']].values
    y = g_sorted['Actual_Sales'].values
    model = LinearRegression()
    model.fit(X, y)
    last_month = X.max()
    # Create future month numbers (wrap around 1-12)
    future_month_nums = np.array([((int(m) - 1) % 12) + 1 for m in range(int(last_month)+1, int(last_month)+1+future_steps)]).reshape(-1,1)
    preds = model.predict(future_month_nums)
    for idx, m in enumerate(future_month_nums.flatten()):
        results.append({
            'Customer': cust,
            'Month_Num': int(m),
            'Predicted_Sales': float(round(preds[idx], 2))
        })

forecast_df = pd.DataFrame(results)
os.makedirs('../data', exist_ok=True)
forecast_df.to_csv('../data/forecasted_sales.csv', index=False)
forecast_df.head()


In [ ]:
# Combine actual and forecast for Tableau and export
combined = df[['Year','Month','Month_Num','Customer','Actual_Sales']].copy()
combined = combined.rename(columns={'Actual_Sales':'Sales'})
forecast_rows = forecast_df.copy()
forecast_rows['Year'] = 'Forecast'
forecast_rows['Month'] = forecast_rows['Month_Num'].apply(lambda x: pd.to_datetime(str(x), format='%m').strftime('%b'))
forecast_rows['Sales'] = forecast_rows['Predicted_Sales']
forecast_rows = forecast_rows[['Year','Month','Month_Num','Customer','Sales']]
combined2 = pd.concat([combined, forecast_rows], ignore_index=True)
combined2.to_csv('../data/sales_for_tableau.csv', index=False)
combined2.tail()
